In [1]:
myDataset={}

myDataset["dev"]={}
myDataset["dev"]["ar"]={}
myDataset["dev"]["ar"]["data"]=[]
myDataset["dev"]["ar"]["label"]=[]
myDataset["dev"]["en"]={}
myDataset["dev"]["en"]["data"]=[]
myDataset["dev"]["en"]["label"]=[]
myDataset["dev"]["ru"]={}
myDataset["dev"]["ru"]["data"]=[]
myDataset["dev"]["ru"]["label"]=[]
myDataset["dev"]["tr"]={}
myDataset["dev"]["tr"]["data"]=[]
myDataset["dev"]["tr"]["label"]=[]

# 创建训练数据集  ar,en,ru,tr各5k
myDataset["train1"]={}
myDataset["train1"]["ar"]={}
myDataset["train1"]["ar"]["data"]=[]
myDataset["train1"]["ar"]["label"]=[]
myDataset["train1"]["en"]={}
myDataset["train1"]["en"]["data"]=[]
myDataset["train1"]["en"]["label"]=[]
myDataset["train1"]["ru"]={}
myDataset["train1"]["ru"]["data"]=[]
myDataset["train1"]["ru"]["label"]=[]
myDataset["train1"]["tr"]={}
myDataset["train1"]["tr"]["data"]=[]
myDataset["train1"]["tr"]["label"]=[]
# 创建训练数据集  ar,en,ru,tr各8k
myDataset["train2"]={}
myDataset["train2"]["ar"]={}
myDataset["train2"]["ar"]["data"]=[]
myDataset["train2"]["ar"]["label"]=[]
myDataset["train2"]["en"]={}
myDataset["train2"]["en"]["data"]=[]
myDataset["train2"]["en"]["label"]=[]
myDataset["train2"]["ru"]={}
myDataset["train2"]["ru"]["data"]=[]
myDataset["train2"]["ru"]["label"]=[]
myDataset["train2"]["tr"]={}
myDataset["train2"]["tr"]["data"]=[]
myDataset["train2"]["tr"]["label"]=[]
# 加载验证集文件
with open("/content/drive/MyDrive/data/dev/dev_ar.txt","r",encoding="utf-8") as file:
  for line in file:
    label,data=line.split("|")
    myDataset["dev"]["ar"]["data"].append(data)
    myDataset["dev"]["ar"]["label"].append(int(label))
with open("/content/drive/MyDrive/data/dev/dev_en.txt","r",encoding="utf-8") as file:
  for line in file:
    label,data=line.split("|")
    myDataset["dev"]["en"]["data"].append(data)
    myDataset["dev"]["en"]["label"].append(int(label))
with open("/content/drive/MyDrive/data/dev/dev_ru.txt","r",encoding="utf-8") as file:
  for line in file:
    label,data=line.split("|")
    myDataset["dev"]["ru"]["data"].append(data)
    myDataset["dev"]["ru"]["label"].append(int(label))
with open("/content/drive/MyDrive/data/dev/dev_tr.txt","r",encoding="utf-8") as file:
  for line in file:
    label,data=line.split("|")
    myDataset["dev"]["tr"]["data"].append(data)
    myDataset["dev"]["tr"]["label"].append(int(label))
# 加载使用chatgpt打标的训练集
with open("/content/drive/MyDrive/data/label_train/labeled_text_by_ChatGPT.txt","r",encoding="utf-8") as file:
  lines=file.readlines()
for i in range(len(lines)):
  temp=lines[i].split("|")
  myDataset["train1"][f"{temp[0]}"]["data"].append(temp[1])
  if "true" in temp[2]:
    myDataset["train1"][f"{temp[0]}"]["label"].append(1)
  else:
    myDataset["train1"][f"{temp[0]}"]["label"].append(0)
# 加载train2训练数据集
with open("/content/drive/MyDrive/data/label_train/output.txt","r",encoding="utf-8") as file:
  for line in file:
    myList=line.split("|")
    if not myList:
      continue
    myDataset["train2"]["en"]["label"].append(int(myList[0]))
    myDataset["train2"]["ar"]["label"].append(int(myList[0]))
    myDataset["train2"]["tr"]["label"].append(int(myList[0]))
    myDataset["train2"]["ru"]["label"].append(int(myList[0]))
    myDataset["train2"]["en"]["data"].append(myList[1])
    myDataset["train2"]["ar"]["data"].append(myList[2])
    myDataset["train2"]["tr"]["data"].append(myList[3])
    myDataset["train2"]["ru"]["data"].append(myList[4])
myDataset["dev"]["all"]={}
myDataset["dev"]["all"]["data"]=myDataset["dev"]["ar"]["data"]+myDataset["dev"]["en"]["data"]+myDataset["dev"]["ru"]["data"]+myDataset["dev"]["tr"]["data"]
myDataset["dev"]["all"]["label"]=myDataset["dev"]["ar"]["label"]+myDataset["dev"]["en"]["label"]+myDataset["dev"]["ru"]["label"]+myDataset["dev"]["tr"]["label"]
myDataset["train1"]["all"]={}
myDataset["train1"]["all"]["data"]=myDataset["train1"]["ar"]["data"]+myDataset["train1"]["en"]["data"]+myDataset["train1"]["ru"]["data"]+myDataset["train1"]["tr"]["data"]
myDataset["train1"]["all"]["label"]=myDataset["train1"]["ar"]["label"]+myDataset["train1"]["en"]["label"]+myDataset["train1"]["ru"]["label"]+myDataset["train1"]["tr"]["label"]
myDataset["train2"]["all"]={}
myDataset["train2"]["all"]["data"]=myDataset["train2"]["ar"]["data"]+myDataset["train2"]["en"]["data"]+myDataset["train2"]["ru"]["data"]+myDataset["train2"]["tr"]["data"]
myDataset["train2"]["all"]["label"]=myDataset["train2"]["ar"]["label"]+myDataset["train2"]["en"]["label"]+myDataset["train2"]["ru"]["label"]+myDataset["train2"]["tr"]["label"]
print(len(myDataset["train2"]["all"]["label"]))
myDataset["train"]={}
myDataset["train"]["all"]={}
myDataset["train"]["all"]["data"]=[]
myDataset["train"]["all"]["label"]=[]
myDataset["train"]["all"]["data"]=myDataset["train1"]["all"]["data"]+myDataset["train2"]["all"]["data"]
myDataset["train"]["all"]["label"]=myDataset["train1"]["all"]["label"]+myDataset["train2"]["all"]["label"]


31992


In [ ]:
# 需要安装的库
!pip install --upgrade pip
!pip install sentencepiece
!pip install datasets
!pip install transformers
!pip install accelerate --upgrade
!pip install transformers[torch]
# 导包
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
import torch
import numpy as np
from sklearn.metrics import classification_report


In [ ]:
MODEL="cardiffnlp/twitter-xlm-roberta-base" #cardiffnlp/twitter-xlm-roberta-base
# 构建分词器，并用到数据集里面
tokenizer=AutoTokenizer.from_pretrained(MODEL,use_fast=True)
dev_all=tokenizer(myDataset["dev"]["all"]["data"],truncation=True,padding=True)
train_all=tokenizer(myDataset["train"]["all"]["data"],truncation=True,padding=True)
import random
# 构建数据集
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __len__(self):
        return len(self.label)
    def __getitem__(self,idx):
        item={key :torch.tensor(value[idx]) for key,value in self.data.items()}
        item["label"]=self.label[idx]
        return item
class RandomMyDataset(torch.utils.data.Dataset):
    def __init__(self, data, label,myList):
        self.data = data
        self.label = label
        self.numbers=myList
    def __len__(self):
        return len(self.label)
    def __getitem__(self,idx):
        idx=self.numbers[idx]
        item={key :torch.tensor(value[idx]) for key,value in self.data.items()}
        item["label"]=self.label[idx]
        return item
val_dataset = MyDataset(dev_all, myDataset["dev"]["all"]["label"])


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2,ignore_mismatched_sizes=True)
model.to("cuda")
BATCH_SIZE_TRAIN=64
BATCH_SIZE_VAL=100
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE_VAL)
EPOCHS=1
ALL_EPOCHS=10
training_args=TrainingArguments(
    output_dir="./result",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE_TRAIN,
    per_device_eval_batch_size=BATCH_SIZE_VAL,
    warmup_steps=100,                         # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,
    learning_rate=1e-5
)

max_res=0
for _ in range(ALL_EPOCHS):
  import random
  numbers = random.sample(range(len(myDataset["train"]["all"]["label"])),len(myDataset["train"]["all"]["label"]) )
  train_dataset=RandomMyDataset(train_all,myDataset["train"]["all"]["label"],numbers)
  # train_dataset=MyDataset(train_all,myDataset["train"]["all"]["label"])
  #选择训练数据集
  trainer = Trainer(
      model=model,                              # the instantiated 🤗 Transformers model to be trained
      args=training_args,                       # training arguments, defined above
      train_dataset=train_dataset,              # training dataset
      eval_dataset=val_dataset               # evaluation dataset
  )
  trainer.train()
  #进行验证集测试准确率
  with torch.no_grad():
    precision=[]
    recall=[]
    for batch in val_loader:
      label=batch["label"]
      del batch["label"]
      batch={key:val.to("cuda") for key,val in batch.items()}
      outputs=model(**batch)
      logits=outputs.logits
      predicted_labels=torch.argmax(logits,dim=1)
      report=classification_report(label,predicted_labels.cpu())
      print(report)
      lines=report.split("\n")
      line=lines[-2]
      print(line)
      row=line.split()
      precision.append(float(row[2]))
      recall.append(float(row[3]))
    res=[]
    with open("res.txt","w") as file:
      for i in range(4):
        print(f"Precision:{precision[i]}\nRecall:{recall[i]}\nScore{(1+0.49)*precision[i]*recall[i]/(0.49*precision[i]+recall[i])}")
        res.append((1+0.49)*precision[i]*recall[i]/(0.49*precision[i]+recall[i]))
        file.write(f"Precision:{precision[i]}\nRecall:{recall[i]}\nScore{(1+0.49)*precision[i]*recall[i]/(0.49*precision[i]+recall[i])}")
    avg_res=sum(res)/len(res)
    print(avg_res)
    if avg_res>max_res:
      max_res=avg_res
      model.save_pretrained('./bm')




Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
10,0.677100
20,0.667300
30,0.660600
40,0.657900
50,0.635200
60,0.639900
70,0.648800
80,0.647300
90,0.619200
100,0.590100


Checkpoint destination directory ./result/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


              precision    recall  f1-score   support

           0       0.57      0.96      0.71        48
           1       0.89      0.33      0.48        52

    accuracy                           0.63       100
   macro avg       0.73      0.64      0.60       100
weighted avg       0.74      0.63      0.59       100

weighted avg       0.74      0.63      0.59       100
              precision    recall  f1-score   support

           0       0.80      0.89      0.84        64
           1       0.76      0.61      0.68        36

    accuracy                           0.79       100
   macro avg       0.78      0.75      0.76       100
weighted avg       0.79      0.79      0.78       100

weighted avg       0.79      0.79      0.78       100
              precision    recall  f1-score   support

           0       0.37      0.76      0.50        34
           1       0.72      0.32      0.44        66

    accuracy                           0.47       100
   macro avg       0

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


...

Step,Training Loss
10,0.243700
20,0.179500
30,0.100700
40,0.074100
50,0.057300
60,0.033300
70,0.009600
80,0.015100
90,0.008700
100,0.014600


Checkpoint destination directory ./result/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


              precision    recall  f1-score   support

           0       0.60      0.94      0.73        48
           1       0.88      0.42      0.57        52

    accuracy                           0.67       100
   macro avg       0.74      0.68      0.65       100
weighted avg       0.75      0.67      0.65       100

weighted avg       0.75      0.67      0.65       100
              precision    recall  f1-score   support

           0       0.82      0.91      0.86        64
           1       0.79      0.64      0.71        36

    accuracy                           0.81       100
   macro avg       0.81      0.77      0.78       100
weighted avg       0.81      0.81      0.80       100

weighted avg       0.81      0.81      0.80       100
              precision    recall  f1-score   support

           0       0.38      0.79      0.51        34
           1       0.76      0.33      0.46        66

    accuracy                           0.49       100
   macro avg       0

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
10,0.223900
